<a href="https://colab.research.google.com/github/abdalla1912mohamed/RAG_LLM_based_RealState_Recommendation_System/blob/main/RAG_RealState_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install pandas --> data Frame manipulation
FAISS --> Facebook similarity Search
Sentence-transformers --> embedding the chunks of data
langchain --> PIPELINE prompt engineering the chains
OPENAI --> the llm model
HUgging_face --> llm and intergration
streamlit --> interface

In [1]:
!pip install pandas faiss-cpu sentence-transformers langchain openai huggingface-hub streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


**DATASET CREATION**
-Creating a dummy dataset mimicking a real-state properties data
**ATTRIBUTES**  
- Description : string
- Price : INT64
- LOCATION : String
- ADDRESS  :
- Compound :
- HAS garden : yes , no
- House Size : INT64
- n of bedrooms
- n of bathrooms

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
NUM_RECORDS = 1000
locations = ['Obour', 'Madinaty', 'Rehab', 'fifth-settlement' , 'Nasr City' , 'Shorouk','El mostakbal city', 'EL Shiekh Zayed' ,  'El Sahel', ' EL GOUNA']
companies = ['SODIC' , 'MIVIDA' , 'PALM HILLS' , "Hassan ALLAM " , "E3mar" , "AZHA" , " AMWAJ" , " MARRASI " , "LOTUS", "BANAFSAG" ]
addresses = ['123', '456','111','222','333','444','555','666','777','888']
descriptions = [
    "A cozy family home with modern design and great amenities.",
    "Luxurious villa with stunning views and spacious interiors.",
    "Affordable apartment in a prime location, close to schools and parks.",
    "Spacious house with a large garden, perfect for families.",
    "Modern condo with state-of-the-art facilities and security.",
    "Contemporary apartment with easy access to transport hubs.",
    "Charming townhouse in a quiet neighborhood with tree-lined streets.",
    "Beachfront property with breathtaking ocean views.",
    "Rustic cottage with a cozy interior and a large backyard.",
    "Penthouse suite with a rooftop pool and panoramic city views."
]

streets = ['1','2','3','4','5','6','7','8','9','10']





RANDOM GENERATORS

In [21]:
# Random functions for generating data
def generate_price():
    return round(random.uniform(100000, 2000000), 2)

def generate_house_size():
    return random.randint(800, 5000)

def generate_num_bedrooms():
    return random.randint(1, 7)

def generate_num_bathrooms():
    return random.randint(1, 5)

def generate_garden():
    return random.choice(['Yes', 'No'])

DATASET_GENERATION

In [23]:
dataset = {
    'Description': [random.choice(descriptions) for _ in range(NUM_RECORDS)],
    'Price': [generate_price() for _ in range(NUM_RECORDS)],
    'Address': [
        f"{random.choice(addresses)} {random.choice(streets)}"
        for _ in range(NUM_RECORDS)
    ],
    'Location': [random.choice(locations) for _ in range(NUM_RECORDS)],
    'company ': [random.choice(companies) for _ in range(NUM_RECORDS)],
    'Has Garden': [generate_garden() for _ in range(NUM_RECORDS)],
    'House Size (m2)': [generate_house_size() for _ in range(NUM_RECORDS)],
    'Number of Bedrooms': [generate_num_bedrooms() for _ in range(NUM_RECORDS)],
    'Number of Bathrooms': [generate_num_bathrooms() for _ in range(NUM_RECORDS)],
}

df = pd.DataFrame(dataset)
df.head()
# Save the dataset to a CSV file
file_name = "real_estate_dataset.csv"
df.to_csv(file_name, index=False)

print(f"Dataset saved to {file_name}")



Dataset saved to real_estate_dataset.csv


EMBEDDING THE DATASET info / Documents
ADDING a new column concatenating the info and another column called embedding for embedding the database

In [36]:
from sentence_transformers import SentenceTransformer
df['info'] = df.apply( lambda row :f"{row['Description']} Location: {row['Location']} Address : {row['Address']} Company : {row['company ']} Garden : {row['Has Garden']} House Size (m2) : {row['House Size (m2)']} Price: {row['Price']} Bedrooms: {row['Number of Bedrooms']} Bathrooms: {row['Number of Bathrooms']} " , axis = 1)
print( df)
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
df['embedding']  = df['info'].apply(lambda x: embedding_model.encode(x))
embeddings = df['embedding'].tolist()
np.save('embeddings.npy', np.array(embeddings))
print( " embeddings are generated")
print(df['embedding'])


                                           Description       Price Address  \
0    Spacious house with a large garden, perfect fo...   934175.06   444 3   
1    Spacious house with a large garden, perfect fo...   913142.79   444 4   
2    Modern condo with state-of-the-art facilities ...   569640.12   333 2   
3    Penthouse suite with a rooftop pool and panora...  1698458.68  222 10   
4    Rustic cottage with a cozy interior and a larg...  1306790.51   456 4   
..                                                 ...         ...     ...   
995  Rustic cottage with a cozy interior and a larg...  1932788.87   888 7   
996  Contemporary apartment with easy access to tra...  1998882.38   555 9   
997  Rustic cottage with a cozy interior and a larg...   169862.05   222 1   
998  Spacious house with a large garden, perfect fo...  1004613.19   111 4   
999  Modern condo with state-of-the-art facilities ...   469263.97   444 3   

      Location       company  Has Garden  House Size (m2)  Numb

In [39]:
import faiss
def vector_search(embeddings):
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  print(index)
  return index

In [72]:
def retrieve_properties(query , top_porperties = 3):
  encoded_query = embedding_model.encode([query])
  embeddings = np.load('embeddings.npy')
  index = vector_search(embeddings)
  distances, indices = index.search(encoded_query, top_porperties)
  results = df.iloc[indices[0]]
  return results


In [73]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load a pre-trained language model (GPT-2 as an example, you can also use GPT-3/4 with OpenAI API)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Function to generate a response using GPT-2
def generate_response(query, retrieved_properties):
    context = f"Query: {query}\nRelevant Properties:\n"
    for idx, row in retrieved_properties.iterrows():
        context += f"- {row['Description']}, {row['Location']}, {row['Price']}.\n"

    # Use the language model to generate a response based on the context
    inputs = tokenizer.encode(context, return_tensors="pt")
    outputs = model.generate(inputs, max_length=160, num_return_sequences=1, no_repeat_ngram_size=2, pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example of querying the system
queries = [
    "I am looking for a villa in OBOUR",
    "I am looking for an apartment in 5th Settlement with a garden",
    "Are there family homes available in Shorouk or Madinaty?",
    "I need an apartment near the beach in SAHEL",
    "Are there any villas in GOUNA?"
]

retrieved_properties = [retrieve_properties(q) for q in queries]
responses = []

# Generate and print responses with a new line for each response
for i in range(len(queries)):
    response = generate_response(queries[i], retrieved_properties[i])
    responses.append(response)
    print(f"Response for query {i+1}:\n{response}\n{'-'*50}\n")  #

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb953075020> >
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb953077840> >
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb953075020> >
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb953077840> >
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb953075020> >
Response for query 1:
Query: I am looking for a villa in OBOUR
Relevant Properties:
- Luxurious villa with stunning views and spacious interiors., Obour, 1208246.98.
- Luxurious villa with stunning views and spacious interiors., Obour, 1834366.26.
- Luxurious villa with stunning views and spacious interiors., Obour, 1986289.06.
The villas are located in the city of Obor, in a small town called Obó, and are situated in an area called Ośrík. The villais

USING LANGACHAIN PROMPTING

In [77]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
import pandas as pd

# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer , max_length = 300)

# Define the PromptTemplate for few-shot learning
prompt_template = """
You are a helpful assistant that provides real estate recommendations based on user queries.
Use the information provided below about properties to generate a response.

Example 1:
Query: "I am looking for a villa in OBOUR"
Relevant Properties:
- Cozy villa, OBOUR, 500000.
Response: "I recommend the cozy villa in OBOUR, which is available for $500,000."

Example 2:
Query: "I am looking for an apartment in 5th Settlement with a garden"
Relevant Properties:
- Modern apartment, 5th Settlement, 300000.
Response: "We have a modern apartment in the 5th Settlement priced at $300,000."

Now, based on the query and the provided properties, generate a response:

Query: {query}
Relevant Properties:
{properties}
Response:
"""

# Define the LangChain pipeline using HuggingFacePipeline
hf_llm = HuggingFacePipeline(pipeline=generator)

# Define the chain
chain = LLMChain(prompt=PromptTemplate(input_variables=["query", "properties"], template=prompt_template), llm=hf_llm)

# Function to generate a better response with LangChain and few-shot prompting
def generate_response_with_few_shots(query, retrieved_properties):
    # Format the retrieved properties as a string
    properties_str = "\n".join([f"- {row['Description']}, {row['Location']}, {row['Price']}" for _, row in retrieved_properties.iterrows()])

    # Run the LangChain pipeline
    return chain.run(query=query, properties=properties_str , max_length = 300)

# Example queries
queries = [
    "I am looking for a villa in OBOUR",
    "I am looking for an apartment in 5th Settlement with a garden",
    "Are there family homes available in Shorouk or Madinaty?",
    "I need an apartment near the beach in SAHEL",
    "Are there any villas in GOUNA?"
]

# Retrieve properties and generate responses for each query
for query in queries:
    retrieved_properties = retrieve_properties(query)
    response = generate_response_with_few_shots(query, retrieved_properties)
    print(f"Response for query: {query}\n{response}\n{'-'*50}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7bb95812d0e0> >


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response for query: I am looking for a villa in OBOUR

You are a helpful assistant that provides real estate recommendations based on user queries. 
Use the information provided below about properties to generate a response.

Example 1:
Query: "I am looking for a villa in OBOUR"
Relevant Properties:
- Cozy villa, OBOUR, 500000.
Response: "I recommend the cozy villa in OBOUR, which is available for $500,000."

Example 2:
Query: "I am looking for an apartment in 5th Settlement with a garden"
Relevant Properties:
- Modern apartment, 5th Settlement, 300000.
Response: "We have a modern apartment in the 5th Settlement priced at $300,000."

Now, based on the query and the provided properties, generate a response:

Query: I am looking for a villa in OBOUR
Relevant Properties:
- Luxurious villa with stunning views and spacious interiors., Obour, 1208246.98
- Luxurious villa with stunning views and spacious interiors., Obour, 1834366.26
- Luxurious villa with stunning views and spacious interior

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response for query: I am looking for an apartment in 5th Settlement with a garden

You are a helpful assistant that provides real estate recommendations based on user queries. 
Use the information provided below about properties to generate a response.

Example 1:
Query: "I am looking for a villa in OBOUR"
Relevant Properties:
- Cozy villa, OBOUR, 500000.
Response: "I recommend the cozy villa in OBOUR, which is available for $500,000."

Example 2:
Query: "I am looking for an apartment in 5th Settlement with a garden"
Relevant Properties:
- Modern apartment, 5th Settlement, 300000.
Response: "We have a modern apartment in the 5th Settlement priced at $300,000."

Now, based on the query and the provided properties, generate a response:

Query: I am looking for an apartment in 5th Settlement with a garden
Relevant Properties:
- Charming townhouse in a quiet neighborhood with tree-lined streets., fifth-settlement, 692631.91
- Affordable apartment in a prime location, close to schools and p

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response for query: Are there family homes available in Shorouk or Madinaty?

You are a helpful assistant that provides real estate recommendations based on user queries. 
Use the information provided below about properties to generate a response.

Example 1:
Query: "I am looking for a villa in OBOUR"
Relevant Properties:
- Cozy villa, OBOUR, 500000.
Response: "I recommend the cozy villa in OBOUR, which is available for $500,000."

Example 2:
Query: "I am looking for an apartment in 5th Settlement with a garden"
Relevant Properties:
- Modern apartment, 5th Settlement, 300000.
Response: "We have a modern apartment in the 5th Settlement priced at $300,000."

Now, based on the query and the provided properties, generate a response:

Query: Are there family homes available in Shorouk or Madinaty?
Relevant Properties:
- A cozy family home with modern design and great amenities., Shorouk, 1371394.52
- Charming townhouse in a quiet neighborhood with tree-lined streets., Shorouk, 1284538.7
- A

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response for query: I need an apartment near the beach in SAHEL

You are a helpful assistant that provides real estate recommendations based on user queries. 
Use the information provided below about properties to generate a response.

Example 1:
Query: "I am looking for a villa in OBOUR"
Relevant Properties:
- Cozy villa, OBOUR, 500000.
Response: "I recommend the cozy villa in OBOUR, which is available for $500,000."

Example 2:
Query: "I am looking for an apartment in 5th Settlement with a garden"
Relevant Properties:
- Modern apartment, 5th Settlement, 300000.
Response: "We have a modern apartment in the 5th Settlement priced at $300,000."

Now, based on the query and the provided properties, generate a response:

Query: I need an apartment near the beach in SAHEL
Relevant Properties:
- Beachfront property with breathtaking ocean views., El Sahel, 1312882.01
- Beachfront property with breathtaking ocean views., El Sahel, 750880.32
- Beachfront property with breathtaking ocean views.

TESTING AND EVALUATING THE MODEL